This notebooks shows an example of using vflow on the Fashion-MNIST dataset using deep neural networks. It requires installing pytorch and torchvision (`pip install torch torchvision`).

In [1]:
%load_ext autoreload
%autoreload 2
from vflow import Vset, init_args, build_vset, dict_to_df
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from functools import partial

In [2]:
# load data
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, fc1=512, fc2=512):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, fc1),
            nn.ReLU(),
            nn.Linear(fc1, fc2),
            nn.ReLU(),
            nn.Linear(fc2, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

def train(model, dataloader, loss_fn, epochs, **kwargs):
    # initialize model with **kwargs
    if isinstance(model, type):
        model = model(**kwargs)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    size = len(dataloader.dataset)
    for t in range(epochs):
        # print(f"Epoch {t+1}\n-------------------------------")
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            # Compute prediction error
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch % 100 == 0:
                loss, current = loss.item(), batch * len(X)
                # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return model


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    return correct

Using cpu device


In [5]:
# we can use ray to distribute computation
import ray
ray.init(num_cpus=4) # optionally might add _temp_dir='tmp'

2022-01-07 12:01:52,687	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.83',
 'raylet_ip_address': '192.168.1.83',
 'redis_address': '192.168.1.83:6379',
 'object_store_address': '/tmp/ray/session_2022-01-07_12-01-50_303116_408484/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-01-07_12-01-50_303116_408484/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-01-07_12-01-50_303116_408484',
 'metrics_export_port': 61025,
 'node_id': '8e9f3e00e78477aaf6f6137467dee0528daeb0c5531e66eb0135b588'}

In [6]:
batch_size = 64
loss_fn = nn.CrossEntropyLoss()

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
train_data, test_data = init_args((train_dataloader, test_dataloader), names=['train_data', 'test_data'])

# fit neural network
modeling_set = build_vset('modeling', train, NeuralNetwork, param_dict={'fc1': [256, 512], 'fc2': [256, 512]}, is_async=True, loss_fn=loss_fn, epochs=5)
modeling_set.fit(train_data)

In [7]:
test_nn = partial(test, loss_fn=loss_fn)
hard_metrics_set = Vset(name='hard_metrics', modules=[test_nn], module_keys=["acc"])
hard_metrics = hard_metrics_set.evaluate(test_data, modeling_set.out)

df = dict_to_df(hard_metrics, param_key='modeling')
df

,init-modeling,init-modeling,fc1-modeling,fc2-modeling,hard_metrics,out
0,test_data,train_data,256,256,acc,0.4976
1,test_data,train_data,256,512,acc,0.5511
2,test_data,train_data,512,256,acc,0.5707
3,test_data,train_data,512,512,acc,0.3181


In [8]:
ray.shutdown()